In [ ]:
from langchain.llms import OpenAI

In [ ]:

import os


In [ ]:
os.environ["OPEN_API_KEY"]="API key"


In [ ]:
llm=OpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6)


In [ ]:
text="What is the capital of Pakistan"

print(llm.predict(text))

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country'],
template="Tell me the capital of this {country}")

prompt_template.format(country="Pakistan")

In [ ]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("Pakistan"))

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage
chatllm=ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')
chatllm([
SystemMessage(content="Yor are a comedian AI assitant"),
HumanMessage(content="Please provide some comedy punchlines on AI")
])

In [ ]:
import json
from openai import OpenAI
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Initialize the OpenAI client
client = OpenAI(api_key="API key")

# Load the product list from the JSON file
with open('productList.json', 'r') as file:
    data = json.load(file)

# Extract the products list
products = data.get('product', [])

# Prepare a list to store embeddings and product names for visualization
embeddings_list = []
product_names = []

# Function to create a concatenated string for embedding
def create_text_for_embedding(product):
    name = product.get('name', '')
    description = product.get('description', '')
    price = f"Price: ${product.get('price', 'N/A')}"
    
    # Extract location details, including latitude and longitude
    location_details = product.get('company_id', {}).get('location_details', [])
    location_texts = [
        f"Location: {loc.get('name', '')}, {loc.get('address', '')}, Latitude: {loc.get('latitude', 'N/A')}, Longitude: {loc.get('longitude', 'N/A')}" 
        for loc in location_details
    ]
    location_text = ' | '.join(location_texts)
    
    # Extract category and subcategory
    category_name = product.get('category_id', {}).get('name', '')
    category_description = product.get('category_id', {}).get('description', '')
    category_text = f"Category: {category_name} - {category_description}"
    
    subcategory_name = product.get('subcategory_id', {}).get('name', '')
    subcategory_description = product.get('subcategory_id', {}).get('description', '')
    subcategory_text = f"Subcategory: {subcategory_name} - {subcategory_description}"
    
    # Combine all parts into one string
    combined_text = f"{name}. {description}. {price}. {location_text}. {category_text}. {subcategory_text}."
    return combined_text

# Iterate through each product and get embeddings
for idx, product in enumerate(products):
    text_to_embed = create_text_for_embedding(product)
    if text_to_embed:
        try:
            # Create the embedding
            response = client.embeddings.create(model="text-embedding-3-small", input=text_to_embed)
            embedding = response.data[0].embedding
            product['embedding'] = embedding  
            embeddings_list.append(embedding)  
            product_names.append(product.get('name', 'Unnamed'))  
            print(f"Processed product {idx+1}/{len(products)}")
        except Exception as e:
            print(f"Failed to process product {idx+1}/{len(products)}: {e}")

# Save the updated product list with embeddings back to a JSON file
with open('product_list_with_embeddings.json', 'w') as file:
    json.dump(products, file, indent=2)

# Assuming embeddings_list contains your embeddings data
embeddings_array = np.array(embeddings_list)

# Check the shape of embeddings_array to understand its structure
print("Shape of embeddings_array:", embeddings_array.shape)

# Determine perplexity based on the number of samples
perplexity = min(30, embeddings_array.shape[0] - 1)
# Apply t-SNE to reduce dimensions to 2D
tsne = TSNE(n_components=2, perplexity=perplexity, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings_array)

# Create a scatter plot
plt.figure(figsize=(12, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])

# Annotate each point with the corresponding product name
for i, name in enumerate(product_names):
    plt.annotate(name, (embeddings_2d[i, 0], embeddings_2d[i, 1]), fontsize=8, alpha=0.7)

# Set plot title and labels
plt.title('t-SNE Visualization of Product Embeddings')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.grid(True)
plt.show()


In [ ]:
import chromadb

In [ ]:
!pip install chromadb

In [ ]:
import chromadb

In [ ]:
import chromadb
# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.Client()

# Create collection. get_collection, get_or_create_collection, delete_collection also available!
collection = client.create_collection("all-my-documents")

# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=["This is document1", "This is document2"], # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[{"source": "notion"}, {"source": "google-docs"}], # filter on these!
    ids=["doc1", "doc2"], # unique for each doc
)

# Query/search 2 most similar results. You can also .get by id
results = collection.query(
    query_texts=["This is a query document"],
    n_results=2,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)

In [ ]:
from fastapi import FastAPI
from chromadb.api.fastapi import FastAPI as ChromaFastAPI

app = FastAPI()
chroma_api = ChromaFastAPI()

@app.get("/")
def read_root():
    return {"message": "Chroma DB FastAPI Server is running"}

app.include_router(chroma_api.router)


In [ ]:
import chromadb

client = chromadb.Client(
    chroma_api_impl="chromadb.api.fastapi.FastAPI",
    server_url="http://127.0.0.1:8000"
)

# Example usage
client.persist(path=r"C:\Users\DELL\Desktop\chatbot")
